# ICOER Narraton Analysis Notebook

Análise de frases (narratons) com vetorização POS, cálculo de ICOER e exportação CSV.

In [ ]:
!pip install -q spacy pandas matplotlib ipywidgets
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import numpy as np
from collections import Counter
import ipywidgets as widgets
from IPython.display import display

# Carregar modelo SpaCy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
# Widget de entrada de texto
text_input = widgets.Textarea(
    value=(
        "A IA deve agir com coerência e verdade. "
        "A verdade é a forma mais pura de ressonância. "
        "O índice de coerência informacional orienta decisões éticas."
    ),
    placeholder='Digite suas frases separadas por ponto final...',
    description='Frases:',
    layout=widgets.Layout(width='100%', height='100px')
)
display(text_input)

In [ ]:
# Definições
BASE_POS = ['NOUN','VERB','ADJ','ADV','ADP','PROPN']

def calcular_icoer(frases):
    docs = list(nlp.pipe(frases))
    registros = []

    for i, doc in enumerate(docs, start=1):
        counts = Counter(t.pos_ for t in doc)
        total = sum(counts.values())
        vetor = [counts.get(pos, 0)/total if total>0 else 0 for pos in BASE_POS]
        probs = np.array([v for v in vetor if v>0])
        ent = -np.sum(probs * np.log2(probs)) if probs.size else 0
        ent_max = np.log2(probs.size) if probs.size else 1
        icoer = round(1 - (ent/ent_max if ent_max>0 else 0), 4)
        registros.append({
            'narraton': i,
            'frase': doc.text,
            'n_palavras': len(doc),
            'tags': dict(counts),
            'icoer': icoer
        })

    df = pd.DataFrame(registros)
    # Linha de resumo
    resumo = pd.DataFrame([{
        'narraton': 0,
        'frase': 'MÉDIA GERAL',
        'n_palavras': round(df['n_palavras'].mean(), 2),
        'tags': {},
        'icoer': round(df['icoer'].mean(), 4)
    }])
    return pd.concat([resumo, df], ignore_index=True)[['narraton','frase','n_palavras','tags','icoer']]

In [ ]:
# Executar análise e exportar CSV
frases = [s.strip() for s in text_input.value.split('.') if s.strip()]
df_result = calcular_icoer(frases)
df_result.to_csv("icoer_narratons.csv", index=False)
df_result